## Pulp Scheduling


In [56]:
from dataclasses import dataclass
from copy import deepcopy
from typing import Tuple, List
import pulp

### Classes


In [57]:
@dataclass
class Topic:
    name: str
    hours: float


@dataclass
class Course:
    name: str
    topics: List[Topic]
    final_day: int
    relative_proportion: float

    def total_hours(self) -> float:
        """Sum of all the hours from each topic."""
        total = 0
        for topic in self.topics:
            total += topic.hours
        return total

### Functions


In [58]:
def validate_courses(courses: Tuple[Course], hours_per_day: List[float]) -> None:
    total_proportion = 0
    n_days = len(hours_per_day)

    for course in courses:
        total_proportion += course.relative_proportion

        if course.final_day > n_days:
            raise ValueError(
                f"The final day of {course.name} ({course.final_day}) "
                f"is outside the number of days ({n_days})."
            )

    if total_proportion != 1:
        raise ValueError(f"Relative proportion did not sum to 1 ({total_proportion})")


def perform_course_scale(course:Course, factor: float) -> Course:
    topics = course.topics

    scaled_topics = []
    for topic in topics:
        scaled_hours = topic.hours*factor
        rounded_hours = round(2 * scaled_hours) / 2
        scaled_topic = Topic(topic.name, rounded_hours)
        scaled_topics.append(scaled_topic)
    
    scaled_course = Course(course.name, scaled_topics, course.final_day, course.relative_proportion)
    return scaled_course

def enforce_hours(course:Course, required_hours: float) -> Course:
    matched_course = deepcopy(course)
    topics = matched_course.topics
    current_hours = matched_course.total_hours()

    i = 0
    while not (required_hours - 0.25 < current_hours < required_hours + 0.25):
        index = i % len(topics)
        if current_hours < required_hours:
            topics[index].hours += 0.5 # Increase by 30 minutes if less than required
        else:
            topics[index].hours -= 0.5 # Decrease by 30 minutes if greater than required

        current_hours = matched_course.total_hours()
        i += 1

    return matched_course
    
        

### Testing


In [60]:
# Inputs
# 1)

# Maths
linear_algebra = Topic("Linear Algebra", 1)
geometry = Topic("Geometry", 2)
differential_equations = Topic("Differential Equations", 4)
maths = Course("Maths", [linear_algebra, geometry, differential_equations], 3, 0.3)

# Physics
o_and_w = Topic("Oscillations and Waves", 3)
mechanics = Topic("Mechanics", 3)
quantum_physics = Topic("Quantum Physics", 4)

physics = Course("Physics", [o_and_w, mechanics, quantum_physics], 5, 0.7)

# Creating the schedule
courses = (maths, physics)
hours_per_day = [2, 3, 3, 2, 4, 3]  # Total = 17


# Testing functionality
def scale_courses(courses: List[Course], hours_per_day: List[int]) -> List[Course]:
    validate_courses(courses, hours_per_day)
    # Determining the number of hours required for each course for this schedule
    total_study_hours = sum(hours_per_day)
    hours_split = []
    for i in range(len(courses)-1):
        course = courses[i]
        hours_to_study = round(course.relative_proportion * total_study_hours)
        hours_split.append(hours_to_study)
    
    # Adding the remaining time
    hours_to_study = round(total_study_hours - sum(hours_split))
    hours_split.append(hours_to_study)

    # Performing the rescale
    rescaled_courses = []
    for course, hours_to_study in zip(courses, hours_split):
        rescaling_factor = hours_to_study / course.total_hours()
        rescaled_course = perform_course_scale(course, rescaling_factor)
        rescaled_courses.append(rescaled_course)

    # Enforcing that the number of hours in each course matches the scaled hours
    enforced_courses = []
    for rescaled_course, hours_to_study in zip(rescaled_courses, hours_split):
        enforced_course = enforce_hours(rescaled_course, hours_to_study)
        enforced_courses.append(enforced_course)


    return enforced_courses


scaled_courses = scale_courses(courses, hours_per_day)

print(scaled_courses)

print(scaled_courses[0].total_hours(), scaled_courses[1].total_hours())

[Course(name='Maths', topics=[Topic(name='Linear Algebra', hours=4.5), Topic(name='Geometry', hours=8.5), Topic(name='Differential Equations', hours=17.0)], final_day=3, relative_proportion=0.3), Course(name='Physics', topics=[Topic(name='Oscillations and Waves', hours=21.0), Topic(name='Mechanics', hours=21.0), Topic(name='Quantum Physics', hours=28.0)], final_day=5, relative_proportion=0.7)]
30.0 70.0
